In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import mutual_info_score
import tensorflow as tf

Preprocess the data by encoding categorical variables and removing null values.\

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv')
df['score_text'] = df['score_text'].replace({'High': 2, 'Medium': 1, 'Low': 0})
df['score_text'] = df['score_text'].fillna(0)
df['v_score_text'] = df['v_score_text'].replace({'High': 2, 'Medium': 1, 'Low': 0})
df['v_score_text'] = df['v_score_text'].fillna(0)
df = df[df['race'].isin(['Caucasian', 'African-American'])]
df['race'] = df['race'].replace({'Caucasian': 1, 'African-American': 0})
df.columns

<ipython-input-2-09c551943218>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'] = df['race'].replace({'Caucasian': 1, 'African-American': 0})


Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')


Make a Logistic Regression model that predicts two_year_recidivism without any prejudice remover. Notice that without any prejudice removal, the model predicts recidivism more accurately for Caucasians compared to African Americans.

In [3]:
X = df[['id', 'age', 'juv_fel_count', 'juv_misd_count', 'is_recid', 'decile_score','juv_other_count', 'priors_count', 'v_score_text', 'is_violent_recid', 'race']].copy()
X = X.fillna(0)
Y = df['two_year_recid'].copy()
unfair_model = LogisticRegression(max_iter=1000)
unfair_model.fit(X, Y)

priv = X[X['race'] == 1]
unpriv = X[X['race'] == 0]

priv_pred = unfair_model.predict(priv)
unpriv_pred = unfair_model.predict(unpriv)

accuracy_priv = accuracy_score(Y[X['race'] == 1], priv_pred)
accuracy_unpriv = accuracy_score(Y[X['race'] == 0], unpriv_pred)

print("Accuracy for privileged group (race == 1):", accuracy_priv)
print("Accuracy for unprivileged group (race == 0):", accuracy_unpriv)

Accuracy for privileged group (race == 1): 0.9759576202118989
Accuracy for unprivileged group (race == 0): 0.963474025974026



Store the class Y, the non-sensitive features X, and the sensitive feature S separately.

In [4]:
non_sensitive_features = ['id', 'age', 'juv_fel_count', 'juv_other_count', 'priors_count', 'v_score_text', 'is_violent_recid']
X = df[non_sensitive_features].copy()
Y = df['two_year_recid'].copy()
S = df['race'].copy()
print(X)
print(Y)
print(S)

         id  age  juv_fel_count  juv_other_count  priors_count  v_score_text  \
1         3   34              0                0             0             0   
2         4   24              0                1             4             0   
3         5   23              0                0             1             1   
6         8   41              0                0            14             0   
8        10   39              0                0             0             0   
...     ...  ...            ...              ...           ...           ...   
7207  10994   30              0                0             0             0   
7208  10995   20              0                0             0             2   
7209  10996   23              0                0             0             1   
7210  10997   23              0                0             0             1   
7212  11000   33              0                0             3             0   

      is_violent_recid  
1             

In [27]:
def PRLOSS(unpriv, priv, eta):
    unpriv_float = tf.cast(unpriv, dtype=tf.float32)
    priv_float = tf.cast(priv, dtype=tf.float32)

    n_unpriv = tf.cast(tf.shape(unpriv_float)[0], dtype=tf.float32)
    n_priv = tf.cast(tf.shape(priv_float)[0], dtype=tf.float32)

    n_unpriv = tf.maximum(n_unpriv, 1.0)
    n_priv = tf.maximum(n_priv, 1.0)

    Dxisi = tf.stack([n_priv, n_unpriv], axis=0)

    y_pred_priv = tf.reduce_sum(priv_float)
    y_pred_unpriv = tf.reduce_sum(unpriv_float)

    P_ys_stacked = tf.stack([y_pred_priv, y_pred_unpriv], axis=0)
    P_ys = P_ys_stacked / Dxisi

    P = tf.concat([unpriv_float, priv_float], axis=0)

    P_sum = tf.reduce_sum(P)
    total_samples = tf.cast(tf.size(unpriv_float) + tf.size(priv_float), dtype=tf.float32)
    P_y = P_sum / total_samples

    P_y = tf.maximum(P_y, 1e-12)

    log_P_ys_1 = tf.math.log(P_ys[1])
    log_P_y = tf.math.log(P_y)
    P_s1y1 = log_P_ys_1 - log_P_y

    log_1_minus_P_ys_1 = tf.math.log(1 - P_ys[1])
    log_1_minus_P_y = tf.math.log(1 - P_y)
    P_s1y0 = log_1_minus_P_ys_1 - log_1_minus_P_y

    log_P_ys_0 = tf.math.log(P_ys[0])
    log_P_y = tf.math.log(P_y)
    P_s0y1 = log_P_ys_0 - log_P_y

    log_1_minus_P_ys_0 = tf.math.log(1 - P_ys[0])
    log_1_minus_P_y = tf.math.log(1 - P_y)
    P_s0y0 = log_1_minus_P_ys_0 - log_1_minus_P_y

    P_s1y1 = tf.reshape(P_s1y1, [-1])
    P_s1y0 = tf.reshape(P_s1y0, [-1])
    P_s0y1 = tf.reshape(P_s0y1, [-1])
    P_s0y0 = tf.reshape(P_s0y0, [-1])

    PI_s1y1 = unpriv_float * P_s1y1
    PI_s1y0 = (1 - unpriv_float) * P_s1y0
    PI_s0y1 = priv_float * P_s0y1
    PI_s0y0 = (1 - priv_float) * P_s0y0

    PI = tf.reduce_sum(PI_s1y1) + tf.reduce_sum(PI_s1y0) + tf.reduce_sum(PI_s0y1) + tf.reduce_sum(PI_s0y0)

    return eta * PI


X_train, X_test, S_train, S_test, Y_train, Y_test = train_test_split(X,Y,S, test_size=0.1, random_state=42)

# In Rishabh's code the loss and val_loss were NaN during training,
# so I checked for NaN or infinite values in the data (to ensure data integrity)
print("NaN values in X_train:", np.any(np.isnan(X_train)))
print("NaN values in X_test:", np.any(np.isnan(X_test)))
print("NaN values in Y_train:", np.any(np.isnan(Y_train)))
print("NaN values in Y_test:", np.any(np.isnan(Y_test)))

# Normalize the input features -> zero mean and unit variance
X_train_normalized = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0)
X_test_normalized = (X_test - X_test.mean(axis=0)) / X_test.std(axis=0)


def prediction_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(input_shape,))
    ])
    return model

# Compile the model with the custom loss function
model = prediction_model(X_train.shape[1])
model.compile(optimizer='adam', loss=lambda y_true, y_pred: PRLOSS(y_true, y_pred, eta))

# Train the model with normalized data
model.fit(X_train_normalized, Y_train, epochs=10, batch_size=32, validation_data=(X_test_normalized, Y_test))

NaN values in X_train: False
NaN values in X_test: False
NaN values in Y_train: False
NaN values in Y_test: False
Epoch 1/10
173/173 [==============================] - 2s 3ms/step - loss: 0.0061 - val_loss: 0.0058
Epoch 2/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0050 - val_loss: 0.0047
Epoch 3/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0038 - val_loss: 0.0042
Epoch 4/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0036 - val_loss: 0.0040
Epoch 5/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0028 - val_loss: 0.0038
Epoch 6/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0026 - val_loss: 0.0038
Epoch 7/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0034 - val_loss: 0.0037
Epoch 8/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0025 - val_loss: 0.0037
Epoch 9/10
173/173 [==============================] - 0s 2ms/step - loss: 0.0024 - val